In [ ]:
from snowflake.snowpark import Session
import pandas as pd
import yfinance as yf

# Snowflake connection
connection_parameters = {
    "account": "your_snowflake_account",
    "user": "your_username",
    "password": "your_password",
    "warehouse": "COMPUTE_WH",
    "database": "CRYPTO_DB",
    "schema": "RAW_CRYPTODB"
}

session = Session.builder.configs(connection_parameters).create()

# Fetch Crypto Prices
def fetch_crypto_prices():
    cryptos = ["BTC-USD", "ETH-USD", "DOGE-USD"]
    data = {crypto: yf.download(crypto, start="2020-01-01") for crypto in cryptos}
    
    df_list = []
    for symbol, df in data.items():
        df["symbol"] = symbol
        df = df.reset_index()[["Date", "symbol", "Close"]]
        df.columns = ["date", "symbol", "price"]
        df_list.append(df)
    
    return pd.concat(df_list)

# Load data into Snowflake
df = fetch_crypto_prices()
session.write_pandas(df, "CRYPTO_PRICES", auto_create_table=True)
